# Simple Solar System Simulation using Astropy

This notebook showcases a simple Solar System simulation using [Astropy](https://www.astropy.org/).

The simulation assumes Solar System bodies are point sources with mass, thus collision is not considered. Small bodies like asteroids, comets, and moons are also omitted. Orbital motions are calculated by considering gravitational force between all the bodies.

Adapted from http://www.cyber-omelette.com/2016/11/python-n-body-orbital-simulation.html.

To run this notebook, the following Python packages are required:

* `numpy`
* `astropy`
* `matplotlib` (for plotting only)

In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
from astropy import constants as const
from astropy import units as u
from astropy.table import QTable
from astropy.visualization import quantity_support

# Let matplotlib understand astropy Quantity.
quantity_support()

%matplotlib notebook

In [ ]:
# Define some common compound units.
vel_unit = u.m / u.s
acc_unit = u.m / (u.s * u.s)

In [ ]:
class Body:
    """Class to handle a body in a planetary system.
    
    Parameters
    ----------
    name : str
        Name of the body.
        
    mass : Quantity
        Mass of the body.
        
    location : Quantity
        Starting position of the body as (x, y, z) vector.
    
    velocity : Quantity
        Starting velocity of the body as (x, y, z) vector.
    
    """
    def __init__(self, name, mass, location, velocity):
        self.name = name
        self.mass = mass
        self.location = location
        self.velocity = velocity

    def calculate_single_body_acceleration(self, bodies):
        """Acceleration of self caused by other bodies."""
        acceleration = [0, 0, 0] * acc_unit

        for other in bodies:
            if other is self:
                continue

            diff = self.location - other.location
            r = np.sqrt(np.sum(diff * diff))
            tmp = const.G * other.mass / (r ** 3)
            acceleration = acceleration + tmp * (other.location - self.location)

        return acceleration

    def move(self, bodies, time_step):
        """Update velocity and location based on interaction with other bodies and given time step."""
        acceleration = self.calculate_single_body_acceleration(bodies)
        self.velocity = self.velocity + (acceleration * time_step)
        self.location = self.location + (self.velocity * time_step)    


class System:
    """Class to handle a planetary system.
    
    Parameters
    ----------
    bodies : list of Body
        List of bodies in the system, including the Sun.
    
    time_step : Quantity
        Time step to use for each iteration.
    
    """
    def __init__(self, bodies, time_step=(1 * u.d)):
        self.bodies = bodies
        self.time_step = time_step

    def run_simulation(self, number_of_steps=10000, report_freq=100):
        """Simulate orbital motions."""
        # Create output container for each body:
        # time step, body1(x,y,z), body2(x,y,z), ...
        body_locations_hist = []
        append_body = body_locations_hist.append

        for i in range(1, number_of_steps + 1):
            for target_body in self.bodies:
                target_body.move(self.bodies, self.time_step)

            if i % report_freq == 0:
                append_body([i * self.time_step] + [b.location.to(u.AU) for b in self.bodies])

        return QTable(rows=body_locations_hist, names=['t'] + [b.name for b in self.bodies])

In [ ]:
def plot_output_all(result, max_range=(50 * u.AU)):
    """Plot all orbits at once within the given range."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    
    body_names = result.colnames[1:]
    colors = ['yellow', 'gray', 'salmon', 'green', 'red', 'brown', 'orange', 'aquamarine', 'skyblue', 'peachpuff']

    if len(body_names) != len(colors):
        raise ValueError('Body and color lists mismatched')

    for b, c in zip(body_names, colors):
        locs = result[b]  # (xyz1, xyz2, ...)
        ax.plot(locs[:, 0], locs[:, 1], locs[:, 2], c=c, label=b)

    ax.set_xlim([-max_range, max_range])    
    ax.set_ylim([-max_range, max_range])
    ax.set_zlim([-max_range, max_range])
    ax.legend(bbox_to_anchor=(-0.05, 1), loc='upper right')
    plt.show()

In [ ]:
def plot_output_timelapse(result, time_pause=0.1, max_range=(50 * u.AU)):
    """Time-lapse view of orbital motions within the given range."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')

    body_names = result.colnames[1:]
    colors = ['yellow', 'gray', 'salmon', 'green', 'red', 'brown', 'orange', 'aquamarine', 'skyblue', 'peachpuff']

    if len(body_names) != len(colors):
        raise ValueError('Body and color lists mismatched')

    for row in result:
        ax.cla()

        for b, c in zip(body_names, colors):
            locs = row[b]
            ax.scatter(*locs, c=c, label=b)

        ax.set_xlim([-max_range, max_range])    
        ax.set_ylim([-max_range, max_range])
        ax.set_zlim([-max_range, max_range])
        ax.set_title(f't = {row["t"]}')
        ax.legend(bbox_to_anchor=(-0.05, 1), loc='upper right')
        fig.canvas.draw()
        time.sleep(time_pause)

Next, we define the Solar System bodies. Mass values are taken from Wikipedia. Starting Y-locations are semi-major axis values, also taken from Wikipedia. Starting velocity values are from the original source linked above.

If you are a planet-purist, you can opt to comment out Pluto. If you do that, please also update `colors` in the plotting functions above.

If you wish to create chaos, such as flinging one of the planets toward the Sun, you can also modify the appropriate velocity values.

In [ ]:
# Body data: Name, mass, location (x, y, z), velocity (x, y, z)
sun = Body('Sun', 1 * u.M_sun, [0, 0, 0] * u.AU,  [0, 0, 0] * vel_unit)
mercury = Body('Mercury', 0.055 * u.M_earth, [0, 0.387098, 0] * u.AU, [47000, 0, 0] * vel_unit)
venus = Body('Venus', 0.815 * u.M_earth, [0, 0.723332, 0] * u.AU, [35000, 0, 0] * vel_unit)
earth = Body('Earth', 1 * u.M_earth, [0, 1, 0] * u.AU, [30000, 0, 0] * vel_unit)
mars = Body('Mars', 0.107 * u.M_earth, [0, 1.523679, 0] * u.AU, [24000, 0, 0] * vel_unit)
jupiter = Body('Jupiter', 1 * u.M_jup, [0, 5.2044, 0] * u.AU, [13000, 0, 0] * vel_unit)
saturn = Body('Saturn', 95.159 * u.M_earth, [0, 9.5826, 0] * u.AU, [9000, 0, 0] * vel_unit)
uranus = Body('Uranus', 14.536 * u.M_earth, [0, 19.2184, 0] * u.AU, [6835, 0, 0] * vel_unit)
neptune = Body('Neptune', 17.147 * u.M_earth, [0, 30.07, 0] * u.AU, [5477, 0, 0] * vel_unit)
pluto = Body('Pluto', 0.00218 * u.M_earth, [0, 39.482, 0] * u.AU, [4748, 0, 0] * vel_unit)

# Fling it to the Sun!
# jupiter.velocity = [0, 13000, 0] * vel_unit

# Build list of planets in the simulation, or create your own.
bodies = [sun, mercury, venus, earth, mars,
          jupiter, saturn, uranus, neptune, pluto]

Now, we can run the simulation! Feel free to adjust the time step, number of steps, and reporting frequency. Smaller time step means higher accuracy. The number of steps defines total simulation time. Lower value of reporting frequency means finer grid of sampling.

This is computationally intensive, so as this next cell is running, you can take a break.

To really speed things up, try reducing the number of bodies you include in your simulation. If you do that, please also update `colors` in the plotting functions above.

In [ ]:
ss = System(bodies, time_step=(1 * u.d))
motions = ss.run_simulation(number_of_steps=1000, report_freq=10)

This prints out the first few rows in the sampled output of your simulation.

In [ ]:
motions[:5]

We can visualize the simulation by drawing out the orbits all at once. You can make `max_range` smaller to zoom in, or larger to zoom out.

In [ ]:
plot_output_all(motions, max_range=(18 * u.AU))

We can also visualize the simulation as a time-lapse animation. You can make `max_range` smaller to zoom in, or larger to zoom out.

In [ ]:
plot_output_timelapse(motions, max_range=(18 * u.AU))